# Finding balance and variability of constraint-based models

Here we use `flux_balance_analysis`,
`flux_variability_analysis`, and
`parsimonious_flux_balance_analysis` of `COBREXA.jl` functions to
analyze a toy model of *E. coli*.

If it is not already present, download the model.

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA


model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel

⠀⠈⢀⠀⡀⠀⠀⠀⠀⡠⠂⠀⠀⠀⠀⠈⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⢀⠐⡀⠀⠀⠀⠀⠄
⠀⠐⠀⠀⠀⠀⠀⠀⡠⠂⠀⠀⠀⠀⢰⠱⣀⠀⡄⢐⠀⠀⢀⠀⠀⠀⡂⠄⠔⠁⠰⠀⠠⠀⣆⠀⠄⢠⢀⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠠⠀⠀⠐⠀⠀⠀⠀⠀⠀⢀⠀⠀⠐⠀⠂⠀⠀⠀⠄⠀⠐⠀⢁⠄⠀⠀⠀⠀⠀
⠀⢀⠀⠐⡈⠀⡀⠀⠂⠀⣀⠀⠑⡈⢀⠀⠀⠀⠀⠀⡀⡠⠀⡀⠰⠁⠈⠂⠁⠀⠠⠀⠀⠂⡂⠀⠂⠂⠀⠀
⠠⠀⠐⠀⠂⠀⠀⢀⠀⠀⠀⠀⠊⠀⡐⠊⠐⠀⠀⠀⠀⠀⠐⠀⠂⠀⠀⠐⠀⠀⠀⠀⠀⠁⠃⠠⠀⠁⠐⠀
⠀⠠⠀⡀⠄⠀⠀⠂⠀⠀⠀⠠⠀⠠⠀⠀⠄⠀⠨⠀⠀⠀⠐⠀⠀⠄⢀⠀⠀⠀⠈⠀⠀⠀⠁⠄⠀⠀⠀⠀
⠀⢐⠐⠀⠄⠀⡂⠀⢐⠀⠀⠀⠀⠂⢀⢀⠐⠂⡀⠈⠀⠀⠀⠂⠀⠈⠀⡀⡐⠀⢄⠀⢀⠀⡆⠀⡀⣀⡀⡐
⠀⠈⠀⠀⠀⠀⠀⠐⢂⠀⢀⠀⠈⠀⠀⠀⠀⠀⠠⠀⠀⠠⠀⠀⠀⠈⠂⠀⠀⠀⠄⠐⠐⠀⠁⠀⠀⠑⠁⠀
⠂⠠⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠠⠈⠀⠀⠀⠀⠀⠁⠀⠀⠠⠐⠀⠁⠈⠀⠁⢀⠀⠀⠀⠀⠀⠀⠀⠀⠌⠀
⠀⠀⠂⢨⠀⡀⠀⠐⠁⠐⠀⠐⠊⠀⠀⠀⠀⠀⠀⠀⠀⠀⠢⠒⠈⠐⠐⠁⠂⠀⠀⠀⠄⠓⠕⠂⠃⠁⠀⠐
⠠⠀⠨⠀⠁⠤⠄⠀⠁⡄⠀⠂⠠⠄⢈⠌⠠⠄⠀⢀⠀⠀⠀⠄⠨⠀⡤⠀⢀⠀⢀⠠⠀⠁⡔⠨⠀⠈⠄⠀
⠀⢀⢀⣀⠀⡠⡒⢀⢀⣀⠀⢀⣀⡀⢀⠀⢀⠀⡀⠀⡀⠀⠈⣀⠀⢀⣀⠀⡀⠀⢀⠁⢀⣀⣀⡀⡠⡀⡀⣀
⠀⠄⠀⠀⠀⠀⠀⠀⠀⠂⠁⠀⠀⠀⠀⠀⠀⣠⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀
⢀⠂⠀⠀⠂⠀⠈⠀⠐⠀⠀⠀⠁⠀⠀⠀⡀⠔⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠢⠀⠀⡀⠂⠈⠀⠀⠀⠄
⠀⠐⠀⠀⡂⠀⠂⠀⠀⠀⠒⠐⠄⠂⠐⠀⠘⡀⠀⠠⡂⠃⠀⠂⠄⠂⠀⠀⠀⠀⡀⠀⡀⠀⡂⠂⠀⠀⢀⠀
Number of reactions: 95
Number of metabolites: 72


## Optimization solvers in `COBREXA`

To actually perform any optimization based analysis we need to load an
optimizer. Any [`JuMP.jl`-supported
optimizers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers)
will work. Here, we will use [`Tulip.jl`](https://github.com/ds4dm/Tulip.jl)
to optimize linear programs and
[`OSQP.jl`](https://osqp.org/docs/get_started/julia.html) to optimize quadratic
programs.

In [2]:
using Tulip, OSQP

## Flux balance analysis (FBA)

Most analysis functions come in several variants that produce different types
of output. All of them usually require a model and `JuMP.jl`-compatible
optimizer to work in the model.

In the case of FBA, you may choose from these variants (here using the
`Tulip` optimizer):

In [3]:
vec_soln = flux_balance_analysis_vec(model, Tulip.Optimizer)

95-element Vector{Float64}:
  -0.0
   6.00724956649032
   7.477381918907127
  -5.064375360152338
   0.2234617471432185
  -3.214895030387032
   2.504309432010867
  21.799492758475754
   4.959985078874371
   1.496983802869297
   ⋮
   3.375438217960911e-7
  29.175827202685298
   9.054357964341115e-9
   4.817965631705414e-8
   9.959461594581987e-9
 -21.799492758475754
  -0.0
  -1.4340676616267298e-9
   3.214895030387032

In [4]:
dict_soln = flux_balance_analysis_dict(model, Tulip.Optimizer)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 6.00725
  "R_TPI"         => 7.47738
  "R_SUCOAS"      => -5.06438
  "R_GLNS"        => 0.223462
  "R_EX_pi_e"     => -3.2149
  "R_PPC"         => 2.50431
  "R_O2t"         => 21.7995
  "R_G6PDH2r"     => 4.95999
  "R_TALA"        => 1.49698
  "R_PPCK"        => 5.88317e-8
  "R_EX_lac__D_e" => 2.39394e-9
  "R_PGL"         => 4.95999
  "R_H2Ot"        => -29.1758
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => 22.8098
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -4.76532
  "R_MALt2_2"     => -0.0
  ⋮               => ⋮

## Modifications

Often it is desirable to add a slight modififaction to the problem before
performing analysis, to see e.g. differences of the model behavior caused by
the change introduced.

`COBREXA.jl` supports several modifications by default, which include
changing objective sense, optimizer attributes, flux constraints,
optimization objective, reaction and gene knockouts, and others.

In [5]:
dict_soln = flux_balance_analysis_dict(
    model,
    OSQP.Optimizer;
    modifications = [ # modifications are applied in order
        # this changes the objective to maximize the biomass production
        change_objective("R_BIOMASS_Ecoli_core_w_GAM"),

        # this fixes a specific rate of the glucose exchange
        change_constraint("R_EX_glc__D_e", -12, -12),

        # this knocks out two genes, i.e. constrains their associated reactions to zero.
        knockout(["b0978", "b0734"]), ## the gene IDs are cytochrome oxidase (CYTBD)

        # ignore the optimizer specified above and change it to Tulip
        change_optimizer(Tulip.Optimizer),

        # set a custom attribute of the Tulip optimizer (see Tulip docs for more possibilities)
        change_optimizer_attribute("IPM_IterationsLimit", 110),

        # explicitly tell the optimizer to maximize the new objective
        change_sense(MAX_SENSE),
    ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 7.03277
  "R_TPI"         => 8.90908
  "R_SUCOAS"      => -5.8921
  "R_GLNS"        => 0.270339
  "R_EX_pi_e"     => -3.88931
  "R_PPC"         => 3.02966
  "R_O2t"         => 25.7859
  "R_G6PDH2r"     => 6.11782
  "R_TALA"        => 1.85013
  "R_PPCK"        => 5.26409e-10
  "R_EX_lac__D_e" => 4.37341e-12
  "R_PGL"         => 6.11782
  "R_H2Ot"        => -34.7096
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => 27.0082
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -5.76498
  "R_MALt2_2"     => -0.0
  ⋮               => ⋮

## Flux variability analysis (FVA)

The default FVA in `flux_variability_analysis` returns maximized and
minimized reaction fluxes in a matrix. Here we use the dictionary variant in
flux_variability_analysis_dict, to show how to easily access specific fluxes
from its results.

In [6]:
fva_mins, fva_maxs = flux_variability_analysis_dict(
    model,
    Tulip.Optimizer;
    bounds = objective_bounds(0.99), # the objective function is allowed to vary by ~1% from the FBA optimum
    modifications = [
        change_optimizer_attribute("IPM_IterationsLimit", 500),
        change_constraint("R_EX_glc__D_e", -10, -10),
        change_constraint("R_EX_o2_e", 0.0, 0.0),
    ],
)

(Dict("R_EX_fum_e" => Dict("R_EX_fum_e" => -0.0, "R_ACONTb" => 0.23975681246428646, "R_TPI" => 9.758987664410151, "R_SUCOAS" => -0.0032176953711697304, "R_GLNS" => 0.060200571606221165, "R_EX_pi_e" => -0.7710723486676665, "R_PPC" => 0.648511462789616, "R_O2t" => 4.7620907185810674e-17, "R_G6PDH2r" => 0.09755599563531388, "R_TALA" => -0.004979593188388901…), "R_ACONTb" => Dict("R_EX_fum_e" => -0.0, "R_ACONTb" => 0.22607952399025047, "R_TPI" => 9.751315002264594, "R_SUCOAS" => -5.7476116849648756e-11, "R_GLNS" => 0.062181345572819555, "R_EX_pi_e" => -0.7708580440964182, "R_PPC" => 0.6578793157880826, "R_O2t" => 9.410658208197004e-15, "R_G6PDH2r" => 0.1207478223560313, "R_TALA" => 0.002761437630187624…), "R_TPI" => Dict("R_EX_fum_e" => -0.0, "R_ACONTb" => 0.22607952434490422, "R_TPI" => 9.485804104911805, "R_SUCOAS" => -8.335641656995056e-11, "R_GLNS" => 0.053580994933664666, "R_EX_pi_e" => -0.7708580445468374, "R_PPC" => 0.6004759354089005, "R_O2t" => 5.507347171694764e-15, "R_G6PDH2r" =

In [7]:
fva_maxs["R_EX_ac_e"]["R_EX_ac_e"] # get the maximal acetate exchange flux

8.518549434877563

## Parsimonious flux balance analysis (pFBA)

Parsimonious flux balance analysis (here in
`parsimonious_flux_balance_analysis` finds a unique flux solution
that minimizes the squared sum of fluxes of the system subject, while
maintaining the same objective value as the flux balance analysis solution.
Since we are optimizing a quadratic objective, we also need to switch to a
quadratic optimizer. In this case, OSQP will work. We demonstrate it on the
dictionary-returning variant of pFBA,
`parsimonious_flux_balance_analysis_dict`:

In [8]:
dict_soln = parsimonious_flux_balance_analysis_dict(
    model,
    OSQP.Optimizer;
    modifications = [
        change_optimizer_attribute("verbose", false), # silence the optimizer (OSQP is very verbose by default)
        change_constraint("R_EX_glc__D_e", -12, -12),
    ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.00740519
  "R_ACONTb"      => 6.50489
  "R_TPI"         => 8.90216
  "R_SUCOAS"      => -5.40139
  "R_GLNS"        => 0.246959
  "R_EX_pi_e"     => -3.85014
  "R_PPC"         => 2.9436
  "R_O2t"         => 25.1755
  "R_G6PDH2r"     => 6.27635
  "R_TALA"        => 1.90489
  "R_PPCK"        => -0.00258038
  "R_EX_lac__D_e" => -0.00523843
  "R_PGL"         => 6.27637
  "R_H2Ot"        => -33.9456
  "R_GLNabc"      => 0.011269
  "R_EX_co2_e"    => 26.4102
  "R_EX_gln__L_e" => -0.0112578
  "R_EX_nh4_e"    => -5.67679
  "R_MALt2_2"     => 0.00661143
  ⋮               => ⋮

The function also has the expectable second variant that returns a vector of
solutions, in `parsimonious_flux_balance_analysis_vec`. Here, we
utilize it to show how to use different optimizers for finding the optimum
and for solving the quadratic problem. That may be preferable if the
optimizer qualities differ for the differing tasks. pFBA allows you to
specify `qp_modifications` that are applied after the original optimum is
found, and before the quadratic part of the problem solving begins.

In [9]:
vec_soln = parsimonious_flux_balance_analysis_vec(
    model,
    Tulip.Optimizer; # start with Tulip
    modifications = [
        change_constraint("R_EX_glc__D_e", -12, -12),
        change_optimizer_attribute("IPM_IterationsLimit", 500), # we may change Tulip-specific attributes here
    ],
    qp_modifications = [
        change_optimizer(OSQP.Optimizer), # now switch to OSQP (Tulip wouldn't be able to finish the computation)
        change_optimizer_attribute("verbose", false), # and silence it.
    ],
)

95-element Vector{Float64}:
  -0.006231402533094581
   6.847122967587614
   8.914435875871503
  -5.738183656745528
   0.25363156710733
  -3.8887743417980833
   2.977310372502161
  25.64208341306474
   6.1991009754149715
   1.877264043236841
   ⋮
  -0.0002197051126034427
  34.49592944330335
  -0.0020752886988600542
  -0.0014018715428246171
  -0.002009900868927077
 -25.642083259145206
   0.0161569956400718
   0.004729690379188246
   3.8887743479784547

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*